In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#Creating the dataframe
df = pd.read_csv("/kaggle/input/vnl-men-2023/VNL2023.csv")

In [ ]:
#Visualizing the original dataframe
df

In [ ]:
df.info()

In [ ]:
#Particionando e dispensando atributos que não vão ser úteis
X = df.drop(['Player', 'Country', 'Position', 'Age'], axis=1)
y = df['Position']

In [ ]:
# Encode da variável target
le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42
)

In [ ]:
# 2. Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Previsões e avaliação
rf_predictions = rf_model.predict(X_test)
print("\nRandom Forest Results:")
print(classification_report(y_test, rf_predictions, 
                          target_names=le.classes_))

In [ ]:
# Importância das features
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)


In [ ]:
# 3. Cross-validation
cv_scores = cross_val_score(rf_model, X_scaled, y, cv=5)
print("\nCross-validation scores:", cv_scores)
print("Mean CV score:", cv_scores.mean())

In [ ]:
# 6. Função de predição corrigida
def predict_position(model, scaler, le, player_stats):
    """
    Prevê a posição de um jogador com base em suas estatísticas
    
    Args:
        model: Modelo treinado
        scaler: Scaler ajustado
        le: LabelEncoder ajustado
        player_stats: Dict com estatísticas numéricas do jogador
    """
    # Criar DataFrame com as mesmas colunas do treino
    stats_df = pd.DataFrame([player_stats], columns=X.columns)
    
    # Normalizar dados
    stats_scaled = scaler.transform(stats_df)
    
    # Fazer previsão
    position_encoded = model.predict(stats_scaled)
    position = le.inverse_transform(position_encoded)
    
    # Obter probabilidades
    probabilities = model.predict_proba(stats_scaled)
    
    return position[0], probabilities[0]




In [ ]:
# 5. Exemplo de uso
new_player = {
    'Age': 25,
    'Attack': 0.65,
    'Block': 0.45,
    'Serve': 0.55,
    'Set': 0.30,
    'Dig': 0.40,
    'Receive': 0.50
}

In [ ]:
position, probs = predict_position(rf_model, scaler, le, new_player)
print("\nPredicted position:", position)
print("Probabilities:", dict(zip(le.classes_, probs)))


In [ ]:
def analyze_position_characteristics(df):
    """Analisa características médias por posição"""
    stats_by_position = df.groupby('Position').agg({
        'Age': 'mean',
        'Attack': 'mean',
        'Block': 'mean',
        'Serve': 'mean',
        'Set': 'mean',
        'Dig': 'mean',
        'Receive': 'mean'
    }).round(2)
    
    return stats_by_position

position_stats = analyze_position_characteristics(df)
print("\nMédia de estatísticas por posição:")
print(position_stats)

In [ ]:
# Visualização geral dos dados 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Verificar tipos de dados
print("Tipos de dados:")
print(df.dtypes)

# Verificar valores únicos em cada coluna
for coluna in df.columns:
    print(f"\nValores únicos em {coluna}:")
    print(df[coluna].unique())

# Verificar se há valores não numéricos nas colunas que deveriam ser numéricas
colunas_numericas = ['Age', 'Attack', 'Block', 'Serve', 'Set', 'Dig', 'Receive']
for coluna in colunas_numericas:
    nao_numericos = df[pd.to_numeric(df[coluna], errors='coerce').isna()]
    if len(nao_numericos) > 0:
        print(f"\nValores não numéricos em {coluna}:")
        print(nao_numericos[coluna])

In [ ]:
# Matriz de correlação
plt.figure(figsize=(10, 8))
correlation_matrix = df[['Age', 'Attack', 'Block', 'Serve', 'Set', 'Dig', 'Receive']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação das Características')
plt.show()

In [ ]:
# 2. Box plots para cada característica por posição
plt.figure(figsize=(15, 10))
features = ['Attack', 'Block', 'Serve', 'Set', 'Dig', 'Receive']

for i, feature in enumerate(features, 1):
   plt.subplot(2, 3, i)
   sns.boxplot(x='Position', y=feature, data=df)
   plt.title(f'{feature} por Posição')
   plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 3. Distribuição de idade por posição
plt.figure(figsize=(10, 6))
sns.boxplot(x='Position', y='Age', data=df)
plt.title('Distribuição de Idade por Posição')
plt.xticks(rotation=45)
plt.show()
